In [10]:
import pandas as pd
import re
from os import listdir
from os.path import isfile, join
print(pd.__version__)

0.22.0


In [11]:
mypath = './data/orig_data/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [12]:
# Datasets with "regular" in the filename
regfiles = [k for k in onlyfiles if 'regular' in k]

In [13]:
total_df = pd.DataFrame(columns=['Player'])
for file in regfiles:
    print(file);
    year = re.findall(r'\d+', file)[0]
    new_df = pd.read_excel('./data/orig_data/' + file)
    new_df['Year'] = int(year)
    
    dupes = new_df[new_df['Tm'] == 'TOT']
    
    dropped = new_df.drop_duplicates(keep=False, subset=['Player','Age'])
    
    new_df = dropped.append(dupes).sort_values(by=['Rk'])
    
    total_df = total_df.append(new_df)
        

regular_statspg_2005season_Raw.xlsx
regular_statspg_2006season_Raw.xlsx
regular_statspg_2007season_Raw.xlsx
regular_statspg_2008season_Raw.xlsx
regular_statspg_2009season_Raw.xlsx
regular_statspg_2010season_Raw.xlsx
regular_statspg_2011season_Raw.xlsx
regular_statspg_2012season_Raw.xlsx
regular_statspg_2013season_Raw.xlsx
regular_statspg_2014season_Raw.xlsx


In [14]:
pre_clean = total_df.sort_values(['Player','Year'])

# Cleaning data
# Having 0 three point percentage is just as good as never shooting 3-pointers
pre_clean['3P.1'] = pre_clean['3P.1'].fillna(0)

# Getting traded in a season is a "bad event." We encode this for players with TOT as their team
pre_clean['Traded'] = pre_clean['Tm'] == 'TOT'
pre_clean['Traded'] = [1 if i else 0 for i in pre_clean['Traded']]
pre_clean = pre_clean.drop(columns=['Tm'])

# One-hot encode a player's position
pre_clean = pd.get_dummies(pre_clean, columns=['Pos'])

# Rank is not a useful feature
pre_clean = pre_clean.drop(columns=['Rk'])

# Reset the indices since several rows were dropped
pre_clean = pre_clean.reset_index(drop=True)

pre_clean.to_csv(path_or_buf='./data/computed/total_player_statistics.csv')